In [1]:
import dask.distributed
import pystac_client
import planetary_computer
import stackstac
import shapely
import numpy as np
import pandas as pd
import rasterio
from matplotlib.colors import ListedColormap


In [2]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster()
client = Client(cluster)
print(client.dashboard_link)

http://127.0.0.1:8787/status


In [3]:
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

In [4]:
import leafmap
m = leafmap.Map(center=[13.85, 100.15], zoom=8, height="800px")
m

Map(center=[13.85, 100.15], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_…

In [6]:
polygon = m.user_rois['features'][0]['geometry']
bbox = shapely.geometry.Polygon(polygon["coordinates"][0]).bounds

In [26]:
search = catalog.search(collections = ["io-lulc-annual-v02"],
                        bbox = bbox,
                        datetime = "2023"
                       )

items = search.item_collection()
print(f"Returned {len(items)} Items")
lc_stack = stackstac.stack(
    items[0],
    dtype=np.ubyte,
    fill_value=255,
    # bounds_latlon=bbox,
    sortby_date=False
).squeeze()
    # .assign_coords(
    #     time=pd.to_datetime([item.properties["start_datetime"] for item in items])
    #     .tz_convert(None)
    #     .to_numpy()
    # )
    # .sortby("time")
m.add_geojson(items[0].to_dict(), zoom=12)
stack

Returned 2 Items


<xarray.DataArray 'stackstac-015b10b87656c07a53d1dfc8cb9a32eb' (y: 88695,
                                                                x: 66148)> Size: 6GB
dask.array<getitem, shape=(88695, 66148), dtype=uint8, chunksize=(1024, 1024), chunktype=numpy.ndarray>
Coordinates: (12/16)
    time            datetime64[ns] 8B NaT
    id              <U8 32B '47P-2023'
    band            <U4 16B 'data'
  * x               (x) float64 529kB 1.693e+05 1.693e+05 ... 8.307e+05
  * y               (y) float64 710kB 1.771e+06 1.771e+06 ... 8.843e+05
    proj:shape      object 8B {66148, 88695}
    ...              ...
    io:tile_id      <U3 12B '47P'
    proj:epsg       int64 8B 32647
    proj:bbox       object 8B {1771250.0, 169260.0, 830740.0, 884300.0}
    file:size       int64 8B 110936866
    raster:bands    object 8B {'nodata': 0, 'spatial_resolution': 10}
    epsg            int64 8B 32647
Attributes:
    spec:        RasterSpec(epsg=32647, bounds=(169260.0, 884300.0, 830740.0,...
    crs:         epsg:32647
    transform:   | 10.00, 0.00, 169260.00|\n| 0.00,-10.00, 1771250.00|\n| 0.0...
    resolution:  10.0

In [28]:
lc_stack.coarsen(x=15, y=15, boundary="trim").mean()

<xarray.DataArray 'stackstac-015b10b87656c07a53d1dfc8cb9a32eb' (y: 5913, x: 4409)> Size: 209MB
dask.array<mean_agg-aggregate, shape=(5913, 4409), dtype=float64, chunksize=(68, 68), chunktype=numpy.ndarray>
Coordinates: (12/16)
    time            datetime64[ns] 8B NaT
    id              <U8 32B '47P-2023'
    band            <U4 16B 'data'
  * x               (x) float64 35kB 1.693e+05 1.695e+05 ... 8.304e+05 8.305e+05
  * y               (y) float64 47kB 1.771e+06 1.771e+06 ... 8.845e+05 8.844e+05
    proj:shape      object 8B {66148, 88695}
    ...              ...
    io:tile_id      <U3 12B '47P'
    proj:epsg       int64 8B 32647
    proj:bbox       object 8B {1771250.0, 169260.0, 830740.0, 884300.0}
    file:size       int64 8B 110936866
    raster:bands    object 8B {'nodata': 0, 'spatial_resolution': 10}
    epsg            int64 8B 32647
Attributes:
    spec:        RasterSpec(epsg=32647, bounds=(169260.0, 884300.0, 830740.0,...
    crs:         epsg:32647
    transform:   | 10.00, 0.00, 169260.00|\n| 0.00,-10.00, 1771250.00|\n| 0.0...
    resolution:  10.0

In [ ]:
lc_stack.plot(
    # subplot_kws=dict(projection=ccrs.epsg(epsg)),
    # transform=ccrs.epsg(epsg),
    # cmap=cmap,
    # vmin=vmin,
    # vmax=vmax,
    figsize=(16, 6),
)

In [ ]:
g = merged.plot(col="time")
for ax in g.axes.flat:
    ax.set_axis_off()

In [ ]:
from pystac.extensions.item_assets import ItemAssetsExtension

collection = catalog.get_collection("io-lulc-annual-v02")
ia = ItemAssetsExtension.ext(collection)

x = ia.item_assets["data"]
class_names = {x["summary"]: x["values"][0] for x in x.properties["file:values"]}
values_to_classes = {v: k for k, v in class_names.items()}
class_count = len(class_names)

with rasterio.open(items[0].assets["data"].href) as src:
    colormap_def = src.colormap(1)  # get metadata colormap for band 1
    colormap = [
        np.array(colormap_def[i]) / 255 for i in range(max(class_names.values()) + 1)
    ]  # transform to matplotlib color format

cmap = ListedColormap(colormap)

vmin = 0
vmax = max(class_names.values()) + 1
epsg = merged.epsg.item()

p = merged.plot(
    # subplot_kws=dict(projection=ccrs.epsg(epsg)),
    col="time",
    # transform=ccrs.epsg(epsg),
    cmap=cmap,
    vmin=vmin,
    vmax=vmax,
    figsize=(16, 6),
)
ticks = np.linspace(0.5, vmax - 0.5, vmax - vmin)
labels = [values_to_classes.get(i, "") for i in range(cmap.N)]
p.cbar.set_ticks(ticks, labels=labels)
p.cbar.set_label("Class")
